<a href="https://colab.research.google.com/github/hyesungKomet/rokaf_ai/blob/main/Elice_3_3_Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Computer Vision에서 Object Detection
**Localistation + Classification**

![](https://www.researchgate.net/profile/Jean-Caprace/publication/355467756/figure/fig1/AS:1086085316784128@1635954326131/Difference-between-classification-localization-detection-and-segmentation-Jaiswal-et.png)

이 사진 하나로 설명이 다 될 것 같다  
* Classification은 사진에 고양이가 있는지 여부만 확인한다. (고양이가 있다 / 없다)  
* 객체 감지는 Classification에 Localization이 합쳐진건데, Localization은 고양이가 "어디에" 있는지를 알아내는 것이다. 둘이 합쳐져서 고양이가 있고, 그게 어디인지 보통 bounding box로 표시해준다  
* Object Detection은 위에서 한 객체 감지를 여러 물체에 대해서 적용한 것이다  
* Instance Segmentation은 선을 딴다고 생각하면 된다 구체적인 경계까지 구하는 것!

## 객체 감지 방법

* two-stage: 클래스의 종류와 위치를 "순서대로" 감지한다 classification과 localization을 각각 수행한다고 보면 될 것 같다
  1. 객체로 의심되는 위치부터 감지
  2. 해당 영역의 객체가 무엇인지 감지

  따로 감지하여 인식률은 높지만 시간이 오래 걸림!

* one-stage: 클래스의 종류와 위치를 동시에 감지!  
한번에 다 수행해서 빠르지만 인식률이 상대적으로 낮음  
You Only ~~Live~~Look Once

# IOU(Intersection Over Union)
이미지 인식의 성능에 대한 지표 필요!

![](https://images.velog.io/images/ailab/post/ce316153-ad09-4b6a-941e-540b53d352d2/image.png)

$$IoU = {교집합 \over 합집합}
$$  
  
완전히 겹칠 때 1, 하나도 안겹칠 때(배반사건이라 하나..?) 0이겠지?

## IOU 함수 구현

![](https://cdn-api.elice.io/api-attachment/attachment/1312d7481ae849229d181086e2fa22c0/image.png)

In [ ]:
import numpy as np


def intersection_over_union(box1, box2):
    iou = 0
    # 교집합 부분의 top left 좌표와 bottom right 좌표
    tlx = np.maximum(box1[0], box2[0])
    tly = np.maximum(box1[1], box2[1])
    brx = np.minimum(box1[2], box2[2])
    bry = np.minimum(box1[3], box2[3])
    
    # 교집합의 넒이
    intersection = np.maximum((brx-tlx), 0) * np.maximum((bry-tly), 0)
    # 0과의 최댓값을 구하는 건 좌표들이 이미지 바운더리를 넘었을 수도 있기 때문
    
    # 박스1의 넓이와 박스2의 넓이
    area1 = (box1[2]-box1[0]) * (box1[3]-box1[1])
    area2 = (box2[2]-box2[0]) * (box2[3]-box2[1])
    
    # 두 박스의 넒이를 더한뒤 교집합 영역 넓이를 뺴 합영역
    union = area1+area2-intersection
    
    # 교집합의 넓이와 합영역을 이용해 IoU를 계산 후 반환
    iou = intersection / union
    
    return iou


if __name__ == "__main__":
    # 아래 두 박스는 좌상단 모서리 점과 우하단 모서점으로 표현
    box1 = [100, 100, 170, 180]
    box2 = [130, 140, 250, 300]
    
    # 완성한 함수를 호출하여 소수점 다섯째 자리에서 반올림하여 값을 출력
    iou = intersection_over_union(box1, box2)
    print(round(iou, 5))

# Scoring
* TP(True Positive): True로 예측한게 실제로도 정답 - 맞음!
* TN(True Negative): False로 예측한게 실제로도 오답 - 맞음!
* FP(False Positive): True로 예측한게 실제로는 오답 - 틀림!
* FN(False Negative): False로 예측해서 틀림 실제로는 정답 - 틀림!

뒤에 P, N이 모델이 True, False로 예측한거고 앞이 예측한게 맞는지 틀린지다. T면 예측한게 맞는거고 F면 예측한게 틀린거다. FN이라고 False를 예측했고 실제로도 False가 아니고, False로 예측한게 틀린거니까 실제로는 True였던거다. 헷갈리지 않게 주의하자!

## Precision
: 모델의 출력이 얼마나 정확한 지의 척도  
모델이 예측한 결과의 Positive 결과가 얼마나 정확한지  
 == 모델이 맞다고 예측한 것 중 실제 정답이 어느정도인지  
 == 모델이 인식한 객체 중 실제로 그 위치에 있는 객체가 어느정도인지  

$$ Precision = {TP \over TP+FP}
= {True\;detections \over whole\; detections\; of\; the\; model}
$$

ex) 날씨가 맑다고 예측했을 때 실제로 맑았는지

## Recall
: 모델의 출력이 얼마나 Positive 값들을 잘 찾는지의 척도
실제로 True인 것 중에 모델이 얼마나 정답을 잘 맞췄는지

$$ Recall = {TP \over TP+FN}
= {detected True \over total\; number\; of\; existing\; True}
$$

ex) 실제로 맑았던 날 중 모델이 맑다고 얼마나 맞췄는지

Precision과 Recall은 관점의 차이! FP를 줄이기 위해 극단적으로 확실한 경우만 맑다고 예측해버리면 30일 중 2일만 맞춰버릴 수도 있다. 이 때 Precision은 높지만 올바른 모델이라고 할 수 있나? 없지! --> Precision과 Recall을 적절히 사용해야 함!

대표적인 예시인 암세포를 살펴보자!  
TP: 암세포라고 예측했고 실제로도 양성이다  

$$ Precision = {TP \over TP+FP} 
= {양성을\; 모델이 \;맞춘 \;경우 \over 모델이\; 양성으로\; 예측한\; 모든\; 경우}
$$  
$$ Recall = {TP \over TP+FN}
= {양성을\; 모델이 \;맞춘\; 경우 \over 실제로\;양성인\; 전체\; 경우}
$$

## Precision-Recall Trade-off
Precision과 Recall은 Win-win 관계가 아닌 Trade-off 관계이다. 하나가 늘면 하나가 줄어든다는 뜻. 벤 다이어그램으로 보면 좀 이해가 된다. 정해진 영역을 가지고, 분모, 분자에 공통인 인자를 가지다 보니 하나가 늘면 하나가 줄고 그런 식이다. 극단적으로 모델이 다 True라고만 예측해버린다고 예를 들어보자. 그럼 FN, TN이 사라지고 그만큼 다른 영역을 채울테다. 그 결과 Recall은 TP / TP로 1이 되어 Precision 영역의 부분집합이 되어버린다. 고런 느낌!

## Accuracy
Precision, Recall은 모두 정답을 True로 예측한 경우만을 따지는데 반해 Accuracy는 False를 False로 예측한 경우도 고려해주는 지표이다. 이 경우도 정답이거덩!  

  
$$ Accuracy = {TP+TN} \over {TP+FN+FP+TN}
$$  
but bias를 주의하자! 맑은 날만 무진장 많은 지역의 경우 Accuracy가 높더라도 흐림이나 비에 대한 예측 성능은 떨어지겠지? -> 보완 지표 필요..!(아래!)

## F1
$$ F1 = 2 \times {precision*recall \over precision+recall}
$$
Precision과 Recall의 조화평균이다 
$$ F1 Score = 2 \times {1 \over {1 \over Precision}+{1 \over Recall}}
$$


# mAP(mean Average Precision)
IoU의 문제점

![](https://www.kdnuggets.com/wp-content/uploads/gad_object_detection_map_07.jpg)

위에서 0.2처럼 IoU가 0보다 크기만 하면, 즉 물체가 있다고 인식하고 물체의 영역과 아주아주 조금이라도 겹치면 잘 찾았다고 하는게 맞나?  
이걸로 성능을 잘 나타낼 수 있는건가?  
재현율과 정밀도가 1인건가??

그래서 준비했습니다

* Threshold  
임계값을 줘서 라벨을 맞추더라도 많이 겹치지 않은 건 적절히 필터링한다  
if IoU < threshold: box = False Positive  
else: box = True Positive  
ex) threshold = 0.3일 때  
A는 Positive, 즉 물체라고 인식했으나 IoU값이 threshold값보다 낮기 때문에 False가 된다. 나머지는 Positive이며 True가 되겠지



$$ AP = Recall \times Presicion
$$

![](https://assets-global.website-files.com/5d7b77b063a9066d83e1209c/622fd6accb7cbe1c48a5a49d_GzND3pjzgEYR7NAxqstFwWYnPV16OARMlbjk1IT2PDBB4UEZnr4qXyGzIVQffkVLkdsST41z9TWtTJ5lrlksxR2zq01IcyiDpH8IKlZMQIWA81vT_iv-DaPmdACLg0tSV7Zl3Q_k.png)

## AP 구하기
1. 모델이 예측한 결과 수집, 예측 신뢰 수준에 따라 순위 나열
2. 두번째 열에는 예측한 라벨 또는 정답 여부
3. Confidence Threshold를 정한다
4. Ground Truth와의 IoU가 Threshold를 넘는 경우 예측이 정확하다고 판단

* Recall = 임계값 넘으면서 올바르게 분류한 경우 / 전체 객체 수
* Precision = 임계값 넘으면서 올바르게 분류한 경우 / 임계값 넘는 경우

* AP  
0부터 1까지 0.1씩 임계값을 올리며 AP를 구한 후 전체인 11로 나누어줌
* mAP
11AP를 모든 클래스에 대해 구하여 평균낸 것

![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FdL1R3c%2FbtqSsQREIRf%2FkNMKsDuz3CPPNu2NSbPmD1%2Fimg.png)

Precision-Recall graph
면적이 Average Precision이다 평균인 것은 전체가 1이므로 나눠도 변하지 않?아서 그렇다고 강의에서 그러는데 맞나..? 잘 모르겄다 생각하기 싫다...

## AP 구하기

In [ ]:
# 클래스가 1개이고 올바르게 검출된 물체의 총 개수가 15개인 어떤 데이터셋

# 문제에서 주어진 변수 detection_results는 어떤 객체 감지 알고리즘을 돌려 나온 결과

# 각각의 행이 하나의 감지된 물체를 나타내며 첫 번째 열의 값은 다음을 의미함

# 1이면 올바르게 검출되고 분류된 경우, 즉 True Positive
# 0이면 물체를 검출되었다고 생각했으나 물체가 아닌 경우를 의미
# 두 번째 열은 클래스에 속할 확률

import numpy

# average precision을 반환하는 함수
def average_precision(detection_results, ground_truth):
    ap = 0

    for i in range(detection_results.shape[0]):
        # 임계값을 0.1씩으로 안하고 detection_results를 확률순으로 정렬한 뒤 거기 값을 하나씩 넣는다!
        threshold = detection_results[i, 1]

        detected = detection_results[numpy.where(detection_results[:, 1] >= threshold)]

        TP = detected[numpy.where(detected[:,0] == 1)]

        #precision = TP / (맞았다고 말한 것 중에 실제 맞은 것)
        #recall = TP / ground_truth

        precision = TP.shape[0] / detected.shape[0]
        recall = TP.shape[0] / ground_truth
        ap += (precision * recall)

    return ap


if __name__ == "__main__":
    detection_results = numpy.array([
        # 15개가 올바르게 검출됐다고 하는데 10개만 있으니 나머지 5개는 True이지만 찾지 못한 TN이 된다
        # TN = 5 FN = 0 TP = 7 FP = 3 이 맞나..?
        [1, 0.95],
        [1, 0.91],
        [1, 0.85],
        [1, 0.81],
        [1, 0.78],
        [0, 0.68],
        [1, 0.57],
        [1, 0.45],
        [0, 0.43],
        [0, 0.13],
    ])
    
    # 정의한 함수를 호출한 결과를 소수점 다섯째 자리에서 반올림하여 확인
    ap = average_precision(detection_results, 15)
    print(round(ap, 4))

# NMS(Non-Max Suppression)
object가 존재하는 위치에 score가 상당한 bounding box가 여러개 만들어질 수 있다. 이 때 하나의 정확도 높은 bounding box만을 선택하기 위한 기법

## 과정

1. confidence threshold를 정하여 이 값 이하인 bounding box는 제거한다
2. 남은 bounding box를 confidence score 기준으로 내림차순 정렬한다
3. 가장 처음의(가장 정확도가 높은) bounding box를 기준으로 다른 bounding box와의 IoU를 구한다
4. IoU threshold를 정하여 이 값 이상인 bounding box는 제거한다
- bounding box끼리 많이 겹쳐질수록 동일한 물체를 인식하는거라고 판단하기 때문
5. confidence threshold, IoU threshold를 높이며 bounding box를 계속 줄여나간다

=> confidence threshold가 높을수록, IoU threshold가 낮을수록 더 많이 제거됨

아래 코드에서는 threshold를 0.1씩 기준으로 하지 않고 bounding box의 IoU를 내림차순 정렬하여 그 값들을 하나씩 threshold로 받아왔다

## YOLO에서의 NMS

YOLO에서 모든 output prediction은 [Pc, Bx, By, Bh, Bw] 형태이다
1. Cell마다 Pc <0.6인 bounding box 제거
2. 남은 값 중 Pc 가장 높은 값 선택
3. 해당 값과의 IoU > 0.5인 bounding box들 제거
4. 그 다음 Pc 가장 높은 값 선택하며 과정 반복

## NMS 함수 구현

In [ ]:
import cv2
import numpy


def intersection_over_union(box1, box2):
    # 교집합 부분의 top left 좌표와 bottom right 좌표 계산
    x1 = numpy.maximum(box1[0], box2[0])
    y1 = numpy.maximum(box1[1], box2[1])
    x2 = numpy.minimum(box1[2], box2[2])
    y2 = numpy.minimum(box1[3], box2[3])

    # 교집합의 넒이
    intersection = numpy.maximum(x2 - x1, 0) * numpy.maximum(y2 - y1, 0)

    # 박스1의 넓이와 박스2의 넓이
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

    # 두 박스의 넒이를 더한뒤 교집합 영역 넓이 빼기.
    union = box1_area + box2_area - intersection

    # iou
    iou = intersection / union
    return iou


def non_max_suppression(boxes, iou_threshold, confidence_threshold):
    nms_boxes = None
    
    
    # 박스의 confidence 값들을 내림차순으로 정렬
    boxes = boxes[numpy.where(boxes[:,0]>confidence_threshold)]

    indices = numpy.argsort(-boxes, axis=0)[:,0] 
    #argsort는 default가 올림차순이라 내림차순하려고 -boxes함(이코테에서 본 스킬이다!)

    

    
    
    # IoU를 계산하여 박스 제거
    for i in range(indices.shape[0]):
        for j in range(i+1, indices.shape[0]):
            if boxes[j,0] == -1:
                continue
            iou = intersection_over_union(boxes[indices[i], 1:],
            boxes[indices[j], 1:])

            if iou >= iou_threshold:
                boxes[j,0] = -1 # iou임계값보다 크면 제거
    
    # 제거된 박스 반환
    nms_boxes = boxes[numpy.where(boxes[:,0] != -1)]
    
    return nms_boxes


if __name__ == "__main__":
    box1 = [0.8, 100, 100, 170, 180]
    box2 = [0.9, 130, 140, 250, 300]
    box3 = [0.6, 100, 150, 290, 170]
    box4 = [0.7, 120, 170, 160, 190]
    box5 = [0.5, 110, 110, 290, 290]
    box6 = [0.3, 240, 200, 340, 270]

    boxes = numpy.array([
        box1, box2, box3, box4, box5, box6
    ])
    # 
    nms_boxes = non_max_suppression(boxes, 0.5, 0.3)
    nms_boxes = list(nms_boxes)
    print(len(nms_boxes))


4


# Anchor Box
NMS의 문제점: 

![](https://www.researchgate.net/profile/Kedar-Potdar-2/publication/329217107/figure/fig4/AS:697578261852162@1543327026650/Object-detection-in-a-dense-scene.ppm)

이와 같은 이미지에서 car를 detecting한다고 할 때 person, umbrella, truck 모두 NMS를 하는 과정에서 bounding box가 겹쳐져 있어서 날라갈 수 있다

Anchor Box는 탐지하려는 객체의 모양을 정해놓고 객체가 탐지됐을 때 어떤 anchor box와 유사한지 판단해서 벡터값을 할당한다

![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbzXK1T%2FbtqWRar8dwM%2FgnQxvkDfC64w7Wz6BLacC0%2Fimg.png)

이 사진처럼 사람과 자동차에 대한 Anchor box를 생성해놓으면 bounding box가 겹치더라도 어떤 anchor box와 더 관련이 있는지를 해당 bounding box와 anchor box와 IoU를 해서 더 겹치는 쪽의 객체로 할당해준다!